# Code to explore how to best handle David's data

(Before I write a whole python script!)


In [108]:
import pyvista as pv # for importing the vtp
import numpy as np
import h5py
from scipy.interpolate import splev, splprep

# can i get the matlab thing to work lol
# just skip matlab, try to use h5

# ALSO the centerline might need a spline fit to it

# maybe just start with STE from earlier? Then move into using the new segmentation...
# Don't think I have enough time to make new thing given the ddx_image_mask.m situation...

In [109]:
# Load the VTP file using PyVista
centerline_model = pv.read("Centerline_model_David.vtp")
centerline_warped = pv.read("Centerline warped.vtp")
centerline_w_branches = pv.read("Scene/Centerline model.vtp")
print(centerline_w_branches)

PolyData (0x1522f6da0)
  N Cells:    4
  N Points:   786
  N Strips:   0
  X Bounds:   1.726e+02, 1.928e+02
  Y Bounds:   1.074e+02, 1.880e+02
  Z Bounds:   -2.507e+02, -5.678e+01
  N Arrays:   6


In [110]:
def are_points_equal(pt1, pt2, tol=1e-6):
    """
    Check if two 3D points are the same within a given tolerance.
    pt1, pt2: array-like, shape (3,)
    tol: float, tolerance for equality
    """
    return np.allclose(pt1, pt2, atol=tol)

In [111]:
main_centerline = centerline_w_branches.get_cell(3).points
centerline_warped_points = centerline_warped.points
branch1 = centerline_w_branches.get_cell(0).points
branch2 = centerline_w_branches.get_cell(1).points
branch3 = centerline_w_branches.get_cell(2).points

#main_centerline = centerline_w_branches.points[main_centerline_idx]
#branch1 = centerline_w_branches.points[branch1]
#branch2 = centerline_w_branches.points[branch2]
#branch3 = centerline_w_branches.points[branch3]

# THE CELLS DON'T SHARE POINTS GODDDAMN

In [112]:
#centerline_points = centerline_model.points
tck, u = splprep([main_centerline[:, 0], main_centerline[:, 1], main_centerline[:, 2]])
new_points = splev(u, tck)
first_deriv = splev(u, tck, der=1)

#centerline_model.plot(show_edges=True, show_scalar_bar=False)
new_points = np.array(new_points).T
print(new_points.shape)
first_deriv = np.array(first_deriv).T


# Create a PolyData object from new_points
new_points_polydata = pv.PolyData(new_points, lines=[358] + np.arange(len(new_points)).tolist())

# Save the PolyData object to a .vtp file
new_points_polydata.point_data.set_vectors(first_deriv, "Derivative")

new_points_polydata.save("new_points.vtp")

(358, 3)


In [113]:
#centerline_points = centerline_model.points
tck, u = splprep([centerline_warped_points[:, 0], centerline_warped_points[:, 1], centerline_warped_points[:, 2]])
new_points_warped = splev(u, tck)
first_deriv_warped = splev(u, tck, der=1)

#centerline_model.plot(show_edges=True, show_scalar_bar=False)
new_points_warped = np.array(new_points_warped).T
print(new_points_warped.shape)
first_deriv_warped = np.array(first_deriv_warped).T


# Create a PolyData object from new_points
new_points_polydata_warped = pv.PolyData(new_points_warped, lines=[344] + np.arange(len(new_points_warped)).tolist())

# Save the PolyData object to a .vtp file
new_points_polydata_warped.point_data.set_vectors(first_deriv_warped, "Derivative")

new_points_polydata_warped.save("new_points_warped.vtp")

(344, 3)


In [114]:
def circularity(area, perimeter):
    return 4 * np.pi * area / (perimeter ** 2)

In [115]:
full_model = pv.read("david_vdm.vtp")
extracted_cuts = []
extracted_surfaces = []

for i in range(len(new_points)):
    
    roi = pv.Sphere(center=new_points[i], radius=35)
    
    test_slice = full_model.slice(origin=new_points[i], normal=first_deriv[i])
    test_slice.clear_data()
    
    extracted = test_slice.clip_surface(roi, invert=True)
    extracted_surf = extracted.delaunay_2d()
    
    lengths = extracted.compute_cell_sizes()

    perimeter = np.sum(lengths["Length"])
    extracted.cell_data.set_scalars(np.ones(extracted.n_cells) * perimeter, "Perimeter")
    extracted.cell_data.set_scalars(np.ones(extracted.n_cells) * circularity(extracted_surf.area, perimeter), "Circularity")
    
    extracted_cuts.append(extracted)
    extracted_surfaces.append(extracted_surf)
    
all_extracted = pv.merge(extracted_cuts)
all_surfaces = pv.merge(extracted_surfaces)

all_extracted.save("all_extracted.vtp")
all_surfaces.save("all_surfaces.vtp")

In [116]:
full_model = pv.read("david_vdm_warped.vtp")
extracted_cuts_warped = []
extracted_surfaces_warped = []

for i in range(len(new_points_warped)):
    
    roi = pv.Sphere(center=new_points_warped[i], radius=35)
    
    test_slice = full_model.slice(origin=new_points_warped[i], normal=first_deriv_warped[i])
    test_slice.clear_data()
    
    extracted = test_slice.clip_surface(roi, invert=True)
    extracted_surf = extracted.delaunay_2d()
    
    lengths = extracted.compute_cell_sizes()

    perimeter = np.sum(lengths["Length"])
    extracted.cell_data.set_scalars(np.ones(extracted.n_cells) * perimeter, "Perimeter")
    extracted.cell_data.set_scalars(np.ones(extracted.n_cells) * circularity(extracted_surf.area, perimeter), "Circularity")
    
    extracted_cuts_warped.append(extracted)
    extracted_surfaces_warped.append(extracted_surf)
    
all_extracted = pv.merge(extracted_cuts_warped)
all_surfaces = pv.merge(extracted_surfaces_warped)

all_extracted.save("all_extracted_warped.vtp")
all_surfaces.save("all_surfaces_warped.vtp")

branch 1

In [6]:
branch_list = [branch1, branch2, branch3]

for i, branch in enumerate(branch_list):
    
    # find unique points first
    unique_in_branch = []
    for pt_branch in branch:
        if not any(are_points_equal(pt_branch, pt_main) for pt_main in main_centerline):
            unique_in_branch.append(pt_branch)

    branch = np.array(unique_in_branch)
    print(f"Unique points in branch {i}: {len(unique_in_branch)}")
    
    tck, u = splprep([branch[:, 0], branch[:, 1], branch[:, 2]])
    new_points = splev(u, tck)
    first_deriv = splev(u, tck, der=1)

    new_points = np.array(new_points).T
    first_deriv = np.array(first_deriv).T

    # Create a PolyData object from new_points
    new_points_polydata = pv.PolyData(new_points, lines=[branch.shape[0]] + np.arange(len(new_points)).tolist())

    # Save the PolyData object to a .vtp file
    new_points_polydata.point_data.set_vectors(first_deriv, "Derivative")

    new_points_polydata.save(f"new_{i}_branch.vtp")

Unique points in branch 0: 53
Unique points in branch 1: 80
Unique points in branch 2: 57


Putting everything in one file...

In [12]:
branch_list = [main_centerline, branch1, branch2, branch3]

new_points_total = []
first_deriv_total = []
point_count = np.zeros(len(branch_list), dtype=int)
topology = []

for i, branch in enumerate(branch_list):
    
    # find unique points first
    if i > 0:
        unique_in_branch = []
        for pt_branch in branch:
            if not any(are_points_equal(pt_branch, pt_main) for pt_main in main_centerline):
                unique_in_branch.append(pt_branch)

        branch = np.array(unique_in_branch)
        print(f"Unique points in branch {i}: {len(unique_in_branch)}")
    
    tck, u = splprep([branch[:, 0], branch[:, 1], branch[:, 2]])
    new_points = splev(u, tck)
    first_deriv = splev(u, tck, der=1)

    new_points_total.append(np.array(new_points).T)
    first_deriv_total.append(np.array(first_deriv).T)
    point_count[i] = branch.shape[0]
    
    if i == 0:
        topology.append([int(point_count[i])] + np.arange(point_count[i]).tolist())
    else:
        topology.append([int(point_count[i])] + np.arange(np.sum(point_count[:i]), np.sum(point_count)).tolist())
        print(f"Current branch length: {branch.shape[0]}")


new_points_total = np.concatenate(new_points_total, axis=0)
first_deriv_total = np.concatenate(first_deriv_total, axis=0)
print(point_count)

#topology = [[branch.shape[0]] + np.arange(branch_list[i].shape[0]).tolist() for branch in branch_list]
topology = [item for sublist in topology for item in sublist]
print(topology)

# Create a PolyData object from new_points
all_new_points_polydata = pv.PolyData(new_points_total, lines=topology)

# Save the PolyData object to a .vtp file
all_new_points_polydata.point_data.set_vectors(first_deriv_total, "Derivative")

all_new_points_polydata.save(f"all_splines.vtp")

Unique points in branch 1: 53
Current branch length: 53
Unique points in branch 2: 80
Current branch length: 80
Unique points in branch 3: 57
Current branch length: 57
[358  53  80  57]
[358, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183

In [ ]:
# cross section analysis: I could make the cross sections and then compare the CE vs MIS diameters OR try to use PCA to pick our the minor and major axes?


